In [ ]:
!pip install sentence-transformers qdrant-client

In [ ]:

import pandas as pd
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer



# -------------------
# 1. Load dataset
# -------------------
with open("tickets.csv", "r", encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

with open("tickets_clean.csv", "w", encoding="utf-8") as f:
    for line in lines:
        f.write(line.replace('"', "'"))

df = pd.read_csv("tickets_clean.csv", on_bad_lines="skip")


df = df.rename(columns={
    "body": "body",
    "answer": "resolution_note",
    "priority": "priority"
})

# -------------------
# 2. Connect to Qdrant
# -------------------
# Local instance
# qdrant = QdrantClient("http://localhost:6333")

# Or cloud instance
qdrant = QdrantClient(url="QDRANT_CLUSTER_ENDPOINT", api_key="QDRANT_API_KEY")

collection_name = "tickets"



# -------------------
# 2. Load Hugging Face embedding model
# -------------------
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# output dim = 384
if collection_name not in [c.name for c in qdrant.get_collections().collections]:
   qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# -------------------
# 4. Insert tickets
# -------------------
points = []
for _, row in df.iterrows():
    ticket_id = str(row.get("id", uuid.uuid4()))
    text_for_embedding = f"Ticket {ticket_id}: {row['body']}"

    embedding = embedder.encode(text_for_embedding).tolist()

    payload = {
        "id": ticket_id,
        "title": row.get("title", ""),
        "body": row.get("body", ""),
        "priority": row.get("priority", "medium"),
        "status": row.get("status", "open"),
        "assignee": row.get("assignee", ""),
        "resolution_note": row.get("resolution_note", "")
    }

    points.append(PointStruct(id=ticket_id, vector=embedding, payload=payload))

# qdrant.upsert(collection_name=collection_name, points=points)
batch_size = 500

for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    qdrant.upsert(collection_name=collection_name, points=batch)
    print(f"Inserted {i+len(batch)} / {len(points)} tickets")


print(f"✅ Ingested {len(points)} tickets into Qdrant collection '{collection_name}'")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Inserted 500 / 24132 tickets
Inserted 1000 / 24132 tickets
Inserted 1500 / 24132 tickets
Inserted 2000 / 24132 tickets
Inserted 2500 / 24132 tickets
Inserted 3000 / 24132 tickets
Inserted 3500 / 24132 tickets
Inserted 4000 / 24132 tickets
Inserted 4500 / 24132 tickets
Inserted 5000 / 24132 tickets
Inserted 5500 / 24132 tickets
Inserted 6000 / 24132 tickets
Inserted 6500 / 24132 tickets
Inserted 7000 / 24132 tickets
Inserted 7500 / 24132 tickets
Inserted 8000 / 24132 tickets
Inserted 8500 / 24132 tickets
Inserted 9000 / 24132 tickets
Inserted 9500 / 24132 tickets
Inserted 10000 / 24132 tickets
Inserted 10500 / 24132 tickets
Inserted 11000 / 24132 tickets
Inserted 11500 / 24132 tickets
Inserted 12000 / 24132 tickets
Inserted 12500 / 24132 tickets
Inserted 13000 / 24132 tickets
Inserted 13500 / 24132 tickets
Inserted 14000 / 24132 tickets
Inserted 14500 / 24132 tickets
Inserted 15000 / 24132 tickets
Inserted 15500 / 24132 tickets
Inserted 16000 / 24132 tickets
Inserted 16500 / 24132 ticke

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
from huggingface_hub import notebook_login
notebook_login()
# Base Mistral model
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)

# Load LoRA adapter directly from Hugging Face repo
ft_model = PeftModel.from_pretrained(base_model, "MA9/ticket-bot-lora")

# Build pipeline
rag_pipeline = pipeline("text-generation", model=ft_model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def retrieve_context(query, k=3):
    # 1. Encode query into vector
    query_vector = embedder.encode(query).tolist()

    # 2. Search Qdrant
    results = qdrant.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=k
    )

    # 3. Format context string
    context = ""
    for r in results:
        payload = r.payload
        context += (
            f"Ticket {payload.get('id', 'N/A')} | "
            f"Priority: {payload.get('priority', 'N/A')} | "
            f"Status: {payload.get('status', 'N/A')}\n"
            f"Assignee: {payload.get('assignee', 'N/A')}\n"
            f"Issue: {payload.get('body', '')}\n"
            f"Resolution: {payload.get('resolution_note', '')}\n\n"
        )

    # If no tickets found, return a fallback
    if not context.strip():
        context = "No related tickets found."
    return context

In [ ]:
def rag_answer(query):
    context = retrieve_context(query)

    prompt = f"""
You are a support assistant. Use only the following tickets to answer.

Tickets:
{context}

User query: {query}

Answer in a structured format with ticket ID, priority, status, assignee, and resolution.
If no related ticket is found, reply: "Sorry, I could not find any related ticket in the system."
"""
    out = rag_pipeline(prompt, max_new_tokens=200, temperature=0.2, do_sample=False)
    return out[0]["generated_text"]


In [ ]:
# Example user queries
queries = [
    "What is the issue in ticket ZD-12345?",
    "Who was assigned to the VPN ticket?",
    "Give me all tickets related to Norton 360.",
    "What is the resolution for the ticket with high priority on Mac?"
]

# Loop over queries
for q in queries:
    print(f"\n🔹 User: {q}")
    answer = rag_answer(q)
    print(f"🤖 Bot: {answer}")



🔹 User: What is the issue in ticket ZD-12345?


/tmp/ipython-input-3463620167.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 Bot: 
You are a support assistant. Use only the following tickets to answer.

Tickets:
Ticket 5d1799cc-a29a-4e01-952d-341da735a361 | Priority: Documentation | Status: open
Assignee: 
Issue: Bug
Resolution: Performance

Ticket b01f94e5-b936-416f-852d-2ffed6cd150d | Priority: Troubleshooting | Status: open
Assignee: 
Issue: Bug
Resolution: Integration

Ticket 9531a95b-b498-4519-9716-4f98e756b529 | Priority: Documentation | Status: open
Assignee: 
Issue: Bug
Resolution: Outage



User query: What is the issue in ticket ZD-12345?

Answer in a structured format with ticket ID, priority, status, assignee, and resolution.
If no related ticket is found, reply: "Sorry, I could not find any related ticket in the system."

Answer: I have identified the tickets with the given query. The tickets are related to documentation issues, bugs, and performance problems. The tickets have the IDs 5d1799cc-a29a-4e01-952d-341da735a361, b01f94e5-b936-416f-852d-2ffed6cd150d, and 9531a95b-b498-4519-9716-4f98e7

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 Bot: 
You are a support assistant. Use only the following tickets to answer.

Tickets:
Ticket 85030d98-d8e9-4bdc-81fb-8aa2ebd49d76 | Priority: 400 | Status: open
Assignee: 
Issue: Incident
Resolution: Product Support

Ticket 852c6e55-8388-433a-8b2d-1f0d666b19a2 | Priority: None | Status: open
Assignee: 
Issue: Network
Resolution: Performance

Ticket 094a5df8-8e64-4f98-a839-3370e88e3303 | Priority: 400 | Status: open
Assignee: 
Issue: Incident
Resolution: Product Support



User query: Who was assigned to the VPN ticket?

Answer in a structured format with ticket ID, priority, status, assignee, and resolution.
If no related ticket is found, reply: "Sorry, I could not find any related ticket in the system."

Answer: I have identified the ticket related to the VPN issue. The ticket number is 852c6e55-8388-433a-8b2d-1f0d666b19a2, and the priority is set to 400. The issue is marked as an incident, and the assignee is currently unknown. If you could provide more details about the problem, 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 Bot: 
You are a support assistant. Use only the following tickets to answer.

Tickets:
Ticket dc3a8cc8-9206-4eb3-9c78-948b062e7937 | Priority: en | Status: open
Assignee: 
Issue:  we can provide you with more specific details about your subscription. We will be in touch soon to discuss this further.'
Resolution: Request

Ticket c6138a12-5e9f-4fb5-91d0-dd183ff45fbc | Priority: en | Status: open
Assignee: 
Issue:  please provide details about the products and services. Additional resources are available; we can discuss by phone for a more convenient and tailored guidance.'
Resolution: Request

Ticket c6deef8f-1a06-4337-a731-11f3078df3ca | Priority: en | Status: open
Assignee: 
Issue:  and offer exclusive promotions.'
Resolution: Request



User query: Give me all tickets related to Norton 360.

Answer in a structured format with ticket ID, priority, status, assignee, and resolution.
If no related ticket is found, reply: "Sorry, I could not find any related ticket in the system."

Answe

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🤖 Bot: 
You are a support assistant. Use only the following tickets to answer.

Tickets:
Ticket a6884327-2092-427e-802d-c9c6d69d2cff | Priority: Performance | Status: open
Assignee: 
Issue: high
Resolution: en

Ticket 231bff6e-c431-4e8f-9132-a8715b0c2169 | Priority: Performance | Status: open
Assignee: 
Issue: high
Resolution: en

Ticket e11f3203-5932-4af3-99e4-e884be0f005c | Priority: Performance | Status: open
Assignee: 
Issue: high
Resolution: en



User query: What is the resolution for the ticket with high priority on Mac?

Answer in a structured format with ticket ID, priority, status, assignee, and resolution.
If no related ticket is found, reply: "Sorry, I could not find any related ticket in the system."

Answer: "The tickets with high priority (<ticket_id_1>, <ticket_id_2>, <ticket_id_3>) are currently open and being addressed by the assignee. The issues are related to performance. The current status is open. If you need further assistance, please contact us at <support_email

CHATBOT


In [ ]:
!pip install streamlit sentence-transformers qdrant-client transformers peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 134.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.1 MB/s eta 0:00:00


In [ ]:
!ngrok config add-authtoken NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

# Qdrant
qdrant = QdrantClient(
    url="QDRANT_CLUSTER_ENDPOINT",
    api_key="QDRANT_API_KEY"
)
collection_name = "tickets"
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Model
base_model_id = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype="auto")
model = PeftModel.from_pretrained(base_model, "MA9/ticket-bot-lora-inference", token="HF_TOKEN")

rag_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def retrieve_context(query, k=3):
    query_vector = embedder.encode(query).tolist()
    results = qdrant.search(collection_name=collection_name, query_vector=query_vector, limit=k)
    context = ""
    for r in results:
        payload = r.payload
        context += (
            f"Ticket {payload.get('id','N/A')} | Priority: {payload.get('priority','N/A')} | "
            f"Status: {payload.get('status','N/A')}\n"
            f"Assignee: {payload.get('assignee','N/A')}\n"
            f"Issue: {payload.get('body','')}\n"
            f"Resolution: {payload.get('resolution_note','')}\n\n"
        )
    return context or "No related tickets found."

def rag_answer(query):
    context = retrieve_context(query)
    prompt = f"""
You are a support assistant. Use only the following tickets to answer.

Tickets:
{context}

User query: {query}

Answer with ticket ID, priority, status, assignee, and resolution if available.
"""
    out = rag_pipeline(prompt, max_new_tokens=200, temperature=0.2, do_sample=False)
    full_text = out[0]["generated_text"]

    # Keep only the new model output after the prompt
    answer = full_text[len(prompt):].strip()
    return answer

# Streamlit UI
st.set_page_config(page_title="IT Ticket Assistant", page_icon="💬")
st.title("💬 IT Ticket RAG Assistant")

user_input = st.chat_input("Ask me about any ticket...")
if user_input:
    answer = rag_answer(user_input)
    with st.chat_message("user"):
        st.write(user_input)
    with st.chat_message("assistant"):
        st.write(answer)


Overwriting app.py


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Public URL:", public_url)
!streamlit run app.py --server.port 8501 --server.headless true &


Public URL: NgrokTunnel: "https://7a5171c419e3.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.188.226:8501

2025-09-20 22:30:25.638477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758407425.664159   14186 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758407425.671975   14186 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758407425.691477   14186 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than o